In [13]:
from isyatirimhisse import fetch_data, fetch_financials, visualize_data

In [24]:
import asyncio

async def main():
    veriler = await fetch_data(symbol='OFSYM', start_date='01-01-2023')
    return veriler

async def run():
    sonuc = await main()
    print(sonuc)

    # Diğer işlemler burada yapılabilir.

async def main_wrapper():
    gorev = asyncio.create_task(run())
    await gorev

await main_wrapper()

         Date   OFSYM
0  2023-08-23   48.10
1  2023-08-24   52.90
2  2023-08-25   58.15
3  2023-08-28   63.95
4  2023-08-29   70.30
5  2023-08-31   77.30
6  2023-09-01   85.00
7  2023-09-04   93.50
8  2023-09-05  102.80
9  2023-09-06  113.00
10 2023-09-07  101.70
11 2023-09-08  102.00
12 2023-09-11   91.80
13 2023-09-12   86.60
14 2023-09-13   77.95
15 2023-09-14   70.20
16 2023-09-15   77.10
17 2023-09-18   69.40
18 2023-09-19   68.55
19 2023-09-20   65.50
20 2023-09-21   67.90
21 2023-09-22   69.30
22 2023-09-25   72.80
23 2023-09-26   69.95
24 2023-09-27   70.55
25 2023-09-28   70.35
26 2023-09-29   69.50
27 2023-10-02   74.05
28 2023-10-03   78.85
29 2023-10-04   81.25
